# SQL for AI Projects

## Introduction

**Natural Language Processing Challenge**

In this Jupyter notebook - we'll quickly setup the DuckDB database, get you familiar with this Google Colab setup and then we'll dive into the NLP challenge exercises for the SQL for AI Projects course!

### Challenge Exercises

1. Clean webpage text data
2. Investigate customer review text
3. Implement A/B test framework

### Database Setup

First things first, let's load up our Python libraries and setup access to our database.

Don't worry if you're not familiar with Python - we'll just need to run the very first cell to initialize our SQL instance and there will be clear instructions whenever there is some non-SQL components.


### Getting Started

To execute each cell in this notebook - you can click on the play button on the left of each cell or you could simply hit the `Run all` button on the top of the notebook just below the menu toolbar.

This cell below will help us download and connect to a DuckDB database object within this notebook's temporary environment.

There will also be a few outputs in the same cell including the following:

* An interactive entity relationship diagram for our database is also as an output from the following cell. This will help us visualize all of the database tables and their relevant primary and foreign keys.

In [ ]:
# Initial setup steps
# ====================

# These pip install commands are required for Google Colab notebook environment
!pip install --upgrade --quiet duckdb==1.3.1
!pip install --quiet duckdb-engine==0.17.0
!pip install --quiet jupysql==0.11.1

# Also need to setup Git LFS for large file dowloads
# This helps us to download large files stored on GitHub
!apt-get install git-lfs -y
!git lfs install

# Clone GitHub repo into a "data" folder
!git clone https://github.com/LinkedInLearning/real-world-data-and-AI-challenges-with-SQL-3813163.git data

# Need to change directory into "data" to run download database object
%cd data
!git lfs pull

# Then we need to change directory back up so all our paths are correct!
%cd ..

# Time to import all our Python packages
import duckdb
import textwrap
import pandas as pd
from pathlib import Path
from IPython.display import HTML, display

# Load the jupysql extension to enable us to run SQL code in code cells
%load_ext sql

# We can now set some basic Pandas settings for rendering SQL outputs
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# This is a convenience function to print long strings into multiple lines
# You'll see this in action later on in our tutorial!
def wrap_print(text):
    print(textwrap.fill(text, width=80))

# This is some boilerplate code to help us format printed output with wrapping
HTML("""
<style>
.output pre {
    white-space: pre-wrap;
    word-break: break-word;
}
</style>
""")

# Connecting to DuckDB
# ====================

# Setup the SQL connection
connection = duckdb.connect("data/data.db")
%sql connection

# Run a few test queries using both connections
tables = connection.execute("SHOW TABLES").fetchall()
table_names = [table[0] for table in tables]

preview_counts_list = []
for table_name in table_names:
    try:
        preview_counts_list.append(
            connection.execute(f"""
                SELECT '{table_name}' AS table_name,
                    COUNT(*) AS record_count
                FROM {table_name}""").fetchdf()
        )
    except Exception as e:
        print(f"❌ Could not preview table {table_name}: {e}")
        

print("✅ Database is now ready!")

print("\n📋 Show count of rows from each table in the database:")

# Combine all dataframes in preview_df_list
preview_counts_df = pd.concat(preview_counts_list, ignore_index=True)

display(preview_counts_df)

display(HTML('''
<iframe width="100%" height="600" src='https://dbdiagram.io/e/685279b3f039ec6d36c0c7e9/68527d19f039ec6d36c1813e'> </iframe>
'''
))

# How to Run SQL Queries

Let's quickly see how we can run SQL code in our Jupyter notebook.

In our Colab environment we can run single or multi-line queries. We can also easily save the output of SQL queries as a local Pandas DataFrame object and even run subsequent SQL queries which can interact with these same DataFrame objects.

## Single Line SQL Query

We can use our notebook magic `%sql` at the start of a notebook cell to run a single line of SQL to query our database.

Let's take a look at the first 5 rows from the `locations` table:

In [ ]:
%sql SELECT * FROM locations LIMIT 5;

## Multi-Line SQL Query

We can also run multi-line SQL queries by using a different notebook magic `%%sql` where we now have 2 percentage signs.

We'll apply a filter on our `location` dataset and return 2 columns.

In [ ]:
%%sql
SELECT
  location_name,
  description
FROM locations
WHERE location_id = 1;

## Saving SQL Outputs

By using the `<<` operator, we can assign the result of a SQL query (returned as a Pandas DataFrame) to a named Python variable in the notebook’s scope.

### Single Line Assignment

We can specify the name of the output variable directly after the `%sql` or `%%sql` magic command.

In [ ]:
%sql single_magic_df << SELECT * FROM locations LIMIT 5;

We can now reference the Python variable directly as a Pandas DataFrame

In [ ]:
# Python notebook scope
single_magic_df

We can also use this same variable as a table reference within a DuckDB `SELECT` statement.

In [ ]:
%sql SELECT * FROM single_magic_df;

### Multi-line Assignment

This assignment using `<<` also works with the `%%sql` (multi-line) magic command.

In [ ]:
%%sql multi_magic_df <<
SELECT
  location_name,
  description
FROM locations
WHERE location_id = 1;

In [ ]:
# display the dataframe
multi_magic_df

When referencing the Python variable within DuckDB, we can also use it inside a multi-line SQL query using the `%%sql` magic command.

In [ ]:
%%sql
SELECT *
FROM multi_magic_df;

# 1. Clean Text Data

In this exercise #1 - we’ll clean and prepare the `html_data` column from the locations table so it’s ready for NLP.

Here is an overview of what we will cover in this tutorial:

* Deep dive into using `REGEXP_REPLACE`
* Remove HTML tags
* Clean up newline, whitespace and `&` characters
* Apply advanced find-and-replace using `REGEXP_REPLACE`
* Maintain original document structure

## 1.1 Inspect Raw Data


### 1.1.2 Viewing Raw HTML

Let’s start by looking at a single row — specifically for Yosemite National Park — to see what kind of cleaning is needed.

We’ll use the `.loc` method in Pandas to inspect the raw HTML. In this case, our expression below is how we would implement - “Get the value from the first row of the DataFrame, specifically from the html_data column.”

```python
yosemite_html_example_df.loc[0, "html_data"]
```

In [ ]:
%sql yosemite_html_example_df << SELECT html_data FROM locations WHERE location_id = 1;

# We'll save this variable for use in a later cell
yosemite_raw_html_string = yosemite_html_example_df.loc[0, "html_data"]

print(yosemite_raw_html_string)

### 1.1.2 Inspect Rendered Data

As we can see - there is a lot of cleaning that needs to be done with this!

Let's take a look at how we can print out our HTML and see how it would render on an actual webpage.

In [ ]:
display(HTML(yosemite_raw_html_string))

## 1.2 Removing HTML Tags

After inspecting the HTML code and our rendered data above - a simple solution comes to mind - potentially we can remove all the tags to only keep us the main text contents that we would see when we visit the actual web page generated by the HTML code.

We can employ some regular expressions - also known as **regex** - and use the `REGEXP_REPLACE` SQL function to help us get this done.

### 1.2.1 Introduction to `REGEXP_REPLACE`

We'll get very familiar with the `REGEXP_REPLACE` function as we'll be using it throughout this tutorial.

An example query we will use below is as follows to extract the text data for Yosemite - we'll also store this as a Python variable `yosemite_removed_tags_df` so we can access it again later:

```sql
SELECT
  REGEXP_REPLACE(html_data, '<[^>]+>', '', 'g') AS text_data
FROM locations
WHERE location_id = 1;
```

Below is a simple breadown of the query components is included here:

| Function         | Purpose                                                |
| ---------------- | ------------------------------------------------------ |
| `REGEXP_REPLACE` | Performs regex-based text replacement                  |
| `html_data     ` | The column of string data that we want to adjust       |
| `'<[^>]+>'`      | Matches any HTML tag like `<p>`, `<h2>`, `<ul>` etc    |
| `[ ... ]`        | Defines a character class to match with inside the [ ] |
| `''`             | Replaces matched text with nothing (i.e. deletes them) |
| `'g'`            | Global flag — apply to all matches, not just the first |

In [ ]:
%%sql yosemite_removed_tags_df <<
SELECT
  REGEXP_REPLACE(html_data, '<[^>]+>', '', 'g') AS text_data
FROM locations
WHERE location_id = 1;

In [ ]:
print(yosemite_removed_tags_df)

## 1.3 Removing Newline Characters

It looks like there's a few newline `\n` characters now appear in our transformed `html_data` string.

We can deploy our `REGEXP_REPLACE` function again to make this work to trim our text outputs and remove those newlines from our already transformed `text_data` column.

This time - notice how I'm using the `yosemite_removed_tags_df` as the target for my `SELECT` statement in my SQL query below.

In [ ]:
%%sql yosemite_removed_tags_trimmed_df <<
SELECT
    REGEXP_REPLACE(text_data, '[\n]', '', 'g') AS text_data
FROM yosemite_removed_tags_df;

In [ ]:
yosemite_removed_tags_trimmed_df

### 1.3.1 Pretty Printing Long Strings

We can't really see the entire string when we just display it like we have above - so I've implemented a neat printing function which we can use to see our string in a slightly nicer format.

The only catch is that we'll need to use our `.loc` notation to extract the text data from our Pandas DataFrame Python variable!

In [ ]:
wrap_print(yosemite_removed_tags_trimmed_df.loc[0, "text_data"])

### 1.3.2 Fixing Our Mistakes

Wait a minute...it looks like we have a few more issues!

Our `REGEXP_REPLACE` might have removed our additional newline characters but now it looks like we've squished a few of our words together in the raw text data.

We can remove these by using our trusty `REGEXP_REPLACE` again - but this time we replace the empty string character with a single whitespace.

Let's apply our changes on the same `yosemite_removed_tags_df` Pandas DataFrame we used for our previous SQL query - but we will assign our output to a new variable called `yosemite_removed_tags_and_newlines_df`

**Note** - yes, I know the long variable names seem like a pain...but we have a popular saying "code is usually read many more times than it's written" so you can think of this as the equivalent of "a stitch in time, saves nine" sort of thing!

In [ ]:
%%sql yosemite_removed_tags_and_newlines_df <<
SELECT
    # This time swap out the '' character for ' '
    REGEXP_REPLACE(text_data, '[\n]', ' ', 'g') AS text_data
FROM yosemite_removed_tags_df;

In [ ]:
wrap_print(yosemite_removed_tags_and_newlines_df.loc[0, "text_data"])

## 1.4 Further Text Cleaning

Now we've got more cleaning to do - maybe we'll want to get rid of those pesky little `&amp;` characters and replace them with a single `&` character.

We've also got a few too many whitespace characters here in our text.

Let's apply our changes one at a time - but at some point we will need to think about how we can combine all of these changes in one go from our source `locations` table instead of applying these transformations one at a time.

Let's perform the following transformations:
1. Replace `&amp;` with `&`
2. Replace one or more whitespace character with a single whitespace

### 1.4.1 `REGEXP_REPLACE` With Special Characters

Sometimes when using `REGEXP_REPLACE` we need to be careful with special characters when we are searching for a specific pattern. Try playing around with the `'&amp;'` below and you'll begin to see what a I mean! If you want to use it with the character class definition square brackets - we'll need to use the backslash character `\` to escape important characters.

Note that these days - it's easy enough to ask an AI to assist with your regular expressions - but back in the old day's we needed to always look these up in Google or use specific Regex checking tools like ["I Hate Regex"](https://ihateregex.io/)

In [ ]:
%%sql yosemite_removed_tags_newlines_ampersand_df <<
SELECT
    REGEXP_REPLACE(text_data, '&amp;', '&', 'g') AS text_data
FROM yosemite_removed_tags_and_newlines_df;

In [ ]:
wrap_print(yosemite_removed_tags_newlines_ampersand_df.loc[0, "text_data"])

### 1.4.2 Regular Expression Character Class

We can also use our character classes `[ ]+` to let our `REGEXP_REPLACE` to find and replace 1 or more whitespace characters in a row with a single whitespace.

In [ ]:
%%sql yosemite_removed_tags_newlines_ampersand_spaces_df <<
SELECT
    # We can use [ ]+ to represent 1 or more spaces
    REGEXP_REPLACE(text_data, '[ ]+', ' ', 'g') AS text_data
FROM yosemite_removed_tags_newlines_ampersand_df;

In [ ]:
wrap_print(yosemite_removed_tags_newlines_ampersand_spaces_df.loc[0, "text_data"])

## 1.5 Combining Transformations

So let's say we want to apply all of our changes that we've identified so far in one-shot from the raw `locations` table within our database.

We have the following transformations to apply:

1. Remove HTML tags
2. Replace multiple newline characters with a single space
3. Replace funny ampersand `&amp;` characters

### 1.5.1 Nested `REGEXP_REPLACE`

We can again complete this task using our trust `REGEXP_REPLACE` function!

The only catch here is that we'll need to use our function in a "nested" form to apply these changes one after another - and we'll need to think about the order of how we apply our changes.

One approach I use to better understand the "nesting" behaviour is to always work from **inside-out** - the SQL engine will start from the most nested transformation first before applying the outer nested function.

Let's give this a shot below and store our results into a variable called `yosemite_one_shot_df`

In [ ]:
%%sql yosemite_one_shot_df <<
SELECT
  REGEXP_REPLACE(
    REGEXP_REPLACE(
        # 1. Most inner function for tag cleanup
        REGEXP_REPLACE(html_data, '<[^>]+>', '', 'g'),
        # 2. Now we can clean up newline and all whitespace in one-shot
        # We also remove any \r returns and \t tab characters
        '[\n\r\t\ ]+', ' ', 'g'
    ),
    # 3. Now we can apply our & update
    '&amp;', '&', 'g'
  )
   AS text_data
FROM locations
WHERE location_id = 1;

In [ ]:
wrap_print(yosemite_one_shot_df.loc[0, "text_data"])

## 1.6 Testing Another Example

We've been performing all our transformations so far on the Yosemite location data - let's take a look at another specific example to challenge our SQL skills and clean our data further!

`location_id = 46` contains the LACMA Los Angeles County Museum of Art details.

This will be a good example for us to implement even further data cleansing steps.

Let's firstly print out our example record to see what we're playing with!

In [ ]:
%sql museum_html_example_df << SELECT html_data FROM locations WHERE location_id = 46;

# We'll save this variable for use in a later cell
museum_raw_html_string = museum_html_example_df.loc[0, "html_data"]

print(museum_raw_html_string)

In [ ]:
print(museum_raw_html_string)

In [ ]:
display(HTML(museum_raw_html_string))

### 1.6.1 Removing Brackets

One of the first things I've noticed here is that we'll likely end up with the round brackets or paranethesis around the `(Los Angeles County Museum of Art)`

We can aim to try and remove these brackets and also apply the same exact transformations we've seen with our Yosemite example.

Let's try this first to see if we need to apply further transforms.

We can use our trusty `REGEXP_REPLACE` to remove both the left and right paranthesis characters - however we'll need to be careful with how we apply the backslash `\` to escape these special regular expression chraracters in our function call.

In [ ]:
%%sql museum_transformed_df <<
SELECT
  REGEXP_REPLACE(
    REGEXP_REPLACE(
        # 1. Most inner function for tag cleanup
        REGEXP_REPLACE(html_data, '<[^>]+>', '', 'g'),
        # 2. Now we can clean up newline and all whitespace in one-shot
        # Here we can also add in our \( and \) escaped paranetheses characters
        '[\n\r\t\(\) ]+', ' ', 'g'
    ),
    # 3. Now we can apply our & update
    '&amp;', '&', 'g'
  )
   AS text_data
FROM locations
WHERE location_id = 46;

In [ ]:
wrap_print(museum_transformed_df.loc[0, "text_data"])

### 1.6.1 Removing Specific Tags

This is ALMOST there - but we have one more complication that we'll need to fix up!

The very first line seems to have a repeat in the location name - we can see `LACMA Los Angeles County Museum of Art LACMA Los Angeles County Museum of Art` in our first line of the previous output.

We'll need to inspect our raw HTML to see where this comes from - if we inspect the first few rows from our raw HTML that we had previously - we can see both a `title` and level 1 heading `h1` tag that repeats the location name.

```html
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>LACMA (Los Angeles County Museum of Art)</title>
</head>
<body>
<h1>LACMA (Los Angeles County Museum of Art)</h1>
...
```

### 1.6.2 Common Ordering Mistakes

We should be right to also use our `REGEXP_REPLACE` in a nested fashion to remove this additional `title` tag to remove the duplicate in our transformed text output - however we'll need to be careful with the order of which we apply the `REGEXP_REPLACE` transformations.

If we were to apply this title tag removal **after** we've already removed **all** of our tags - then nothing would happen!

Let's demonstrate this in action before we see how we should fix it - imagine we wanted to apply `<title>` tag removal after all of our original transformations using this regular expression: `(?si)<title>.*?</title>`

The breakdown of what's happening in this regular expression is below:

* (?si) – two inline flags
    + s: single‑line mode, so the dot . matches everything, including newlines
    + i: case‑insensitive, so it will match <title>, <Title>, <TITLE>, etc. 
* <title> – literally matches the opening tag that we're after
* `.*?` – a non‑greedy match of any characters from just after <title> to the earliest possible </title>
* </title> – literally matches the closing `</title> tag


In [ ]:
%%sql museum_transformed_removed_title_df <<
SELECT
  REGEXP_REPLACE(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        # 1. Most inner function for tag cleanup
        REGEXP_REPLACE(html_data, '<[^>]+>', '', 'g'),
        # 2. Now we can clean up newline and all whitespace in one-shot
        # Here we can also add in our \( and \) escaped paranetheses characters
        '[\n\r\t\(\) ]+', ' ', 'g'
      ),
      # 3. Now we can apply our & update
      '&amp;', '&', 'g'
    ),
    # 4. Let's say we put in our <title> removal here...
    '(?si)<title>.*?</title>', '', 'g'
  ) AS text_data
FROM locations
WHERE location_id = 46;

In [ ]:
wrap_print(museum_transformed_removed_title_df.loc[0, "text_data"])

### 1.6.2 Fixing Up The Order

We can still see the repetition right at the beginning of our text!

This is because when we apply the `REGEXP_REPLACE` to remove our HTML tags - we inadvertantly also remove the `<title>` tags we are looking to replace so our follow-up `REGEXP_REPLACE` call doesn't see the data.

Let's try this again - but we'll adjust the order of our replacements a little and store our outputs in `museum_transformed_removed_title_adjusted_order_df` (I know...the names are getting a bit long right?!)


In [ ]:
%%sql museum_transformed_removed_title_adjusted_order_df <<
SELECT
  REGEXP_REPLACE(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        # 1. Let's say we put in our <title> removal here this time!
        REGEXP_REPLACE(html_data, '(?si)<title>.*?</title>', '', 'g'),
        # 2. Now we can remove all of our tags
        '<[^>]+>', '', 'g'
      ),
      # 3. Now we can clean up newline and all whitespace in one-shot
      # Here we can also add in our \( and \) escaped paranetheses characters
      '[\n\r\t\(\) ]+', ' ', 'g'
    ),
    # 4. Finally we can remove the ampersand and we're done!
    '&amp;', '&', 'g'
  ) AS text_data
FROM locations
WHERE location_id = 46;

In [ ]:
wrap_print(museum_transformed_removed_title_adjusted_order_df.loc[0, "text_data"])

## 1.7 Retaining Document Structure

Excellent - we've managed to remove the repetition at the beginning of our museum example!

But there's more we can do!

For traditional NLP - this is probably good enough, we have extracted the raw text and cleaned up most of our tags, additional spaces and fixed the ampersand web-escaped characters.

However - for modern LLMs we can take it further and attempt to retain as much of our original document structure as possible. We can do this by further manipulating our raw text into discrete sections.

For this exercise - we will need to really inspect our raw HTML to see how we might apply a good generalized rule and apply it across all our documents.


### 1.7.1 Identifying What to Retain

If we dive into our raw HTML - we'll be able to see how our level 2 headings might be useful to structure our cleaned text output.

These `<h2> ... </h2>` tags can be used with our `REGEXP_REPLACE` to help retain the structure of the document.

```html
<h2>Summary</h2>
<p>The largest art museum in the western United States, with a collection of nearly 150,000 works spanning the history of art from ancient times to the present.</p>
</section>
<section id="best-time">
<h2>Best Time to Visit</h2>
<p>Spring and fall usually offer mild weather and smaller crowds. Always check local conditions, as climate can vary by elevation.</p>
</section>
```

### 1.7.2 Advanced Find and Replace

For our exercise - let's surround whatever contents are inside the H2 heading with a single pipe character. 

It's a good idea to surround the level 2 contents with a pipe characters `|` before and after the heading text.

We can accomplish this find and replace task using the same `REGEXP_REPLACE` function but this time with a slightly different variation using variable substitution!

We can use the regular expression: `'<h2>|</h2>'` to replace any occurences of `<h2>` or `</h2>` within the `REGEXP_REPLACE` command.

```sql
REGEXP_REPLACE(html_data, '(?si)<h2>(.*?)</h2>', '| \1 |', 'g')
```

Here is a simple breakdown of this regular expression function:


| Component         | Purpose                                                      |
| ----------------- | ------------------------------------------------------------ |
| `(?si)`           | Matches across multiple lines and is case-insensitive        |
| `<h2>`            | Matches the literal opening tag `<h2>`                       |
| `(.*?)`           | Capturing group that contains any character between the tags |
| `</h2>`           | Matches the literal closing of the tag `</h2>`               |
| `'| \1 |'`        | A backreference to the first (and only) capturing group      |
| `'g'`             |  Global flag — apply to all matches, not just the first      |

In [ ]:
%%sql museum_further_transformed_df <<
SELECT
  REGEXP_REPLACE(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        REGEXP_REPLACE(
          # 1. remove title tag contents
          REGEXP_REPLACE(html_data, '(?si)<title>.*?</title>', '', 'g'),
          # 2. surround level 2 contents with | characters
          '(?si)<h2>(.*?)</h2>', '| \1 |', 'g'
        ),
        # 3. clean up - remove all other tags
        '<[^>]+>', '', 'g'
      ),
      # 4. further clean up of whitespace and newlines
      '[\n\r\t\(\) ]+', ' ', 'g'
    ),
    # 5. Finally we can remove the ampersand and we're done!
    '&amp;', '&', 'g'
  ) AS text_data
FROM locations
WHERE location_id = 46;

In [ ]:
wrap_print(museum_further_transformed_df.loc[0, "text_data"])

### 1.7.3 Remove Arbitrary Text

This is very close to perfect - but I've noticed one more step we can take to further clean up our output!

If we look at the end of our text data output - we can see that our ending of the `text_data` field ends with the following:

```text
| Useful Links | View on Google Maps Wikipedia Article
```

It seems that our hyperlink information is removed due to our previous regular expression removing all of the HTML tags. 

We can apply another `REGEXP_REPLACE` at the end of our series of transformations to remove everything from `Useful Links` to the end of the string.

I've also noticed an additional single whitespace character at the beginning of our text-string - so let's also apply a simple `TRIM` function to strip out all leading and trailing whitespace characters also.

This will be our final transformation - so let's store our outputs as the variable `museum_final_transformed_df`

In [ ]:
%%sql museum_final_transformed_df <<
SELECT
  TRIM(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        REGEXP_REPLACE(
          REGEXP_REPLACE(
            REGEXP_REPLACE(
              # 1. remove title tag contents
              REGEXP_REPLACE(html_data, '(?si)<title>.*?</title>', '', 'g'),
              # 2. surround level 2 contents with | characters
              '(?si)<h2>(.*?)</h2>', '| \1 |', 'g'
            ),
            # 3. clean up - remove all other tags
            '<[^>]+>', '', 'g'
          ),
          # 4. further clean up of whitespace and newlines
          '[\n\r\t\(\) ]+', ' ', 'g'
        ),
        # 5. We can remove the ampersand
        '&amp;', '&', 'g'
      ),
      # 6. Finally remove the final useful links / wiki missing links
      # We'll need to escape the pipe character as it's special!
      # The $ denotes the end of the line so we remove everything from | Useful...
      '\| Useful Links.*$', '', 'g'
    )
  ) AS text_data
FROM locations
WHERE location_id = 46;

In [ ]:
wrap_print(museum_final_transformed_df.loc[0, "text_data"])

## 1.8 Apply Transformations to Entire Dataset

This is perfect! Let's now remove our `WHERE` filter and apply this to our entire dataset and we're done for exercise 1!

We can store our outputs as `locations_transformed_df` and we can quickly check that our transformations look alright for the previous records we were checking `location_id = 1` and `location_id = 46` for our Yosemite and Museum examples.

In [ ]:
%%sql locations_transformed_df <<
SELECT
  # we can keep all of our existing locations data here in our final dataset
  locations.*,
  TRIM(
    REGEXP_REPLACE(
      REGEXP_REPLACE(
        REGEXP_REPLACE(
          REGEXP_REPLACE(
            REGEXP_REPLACE(
              # 1. remove title tag contents
              REGEXP_REPLACE(html_data, '(?si)<title>.*?</title>', '', 'g'),
              # 2. surround level 2 contents with | characters
              '(?si)<h2>(.*?)</h2>', '| \1 |', 'g'
            ),
            # 3. clean up - remove all other tags
            '<[^>]+>', '', 'g'
          ),
          # 4. further clean up of whitespace and newlines
          '[\n\r\t\(\) ]+', ' ', 'g'
        ),
        # 5. We can remove the ampersand
        '&amp;', '&', 'g'
      ),
      # 6. Finally remove the final useful links / wiki missing links
      # We'll need to escape the pipe character as it's special!
      # The $ denotes the end of the line so we remove everything from | Useful...
      '\| Useful Links.*$', '', 'g'
    )
  ) AS text_data
FROM locations

In [ ]:
# Let's check out our work!
locations_transformed_df.head()

In [ ]:
# Check Yosemite which is our first row
# Pandas DataFrames are 0-indexed so the first record is the 0th row
wrap_print(locations_transformed_df.loc[0, "text_data"])

In [ ]:
# Check Museum example which is our 46th row - index should be 45
wrap_print(locations_transformed_df.loc[45, "text_data"])

# 2. Customer Reviews

In this part of our tutorial we'll implement some basic NLP techniques using SQL on our customer reviews dataset.

The following NLP techniques will be covered:

* Remove stopwords
* Bag of Words
* Term Frequency
* Document Frequency
* TF-IDF

We'll also use our TF-IDF outputs to answer some simple questions about our user reviews for each tour product.

These exercises aim to help you build this skeptical and curious mindset when it comes to preparing data for AI and machine learning tasks!

**Note:** in practice - we'll routinely use Python for these NLP transformations, however it's still a great exercise to execute a similar process using SQL only.

Currently in `DuckDB` there is no direct natural language processing functions - so we have a perfect challenge to implement some simple traditional NLP techniques using SQL only.

## 2.1 Inspect Reviews Data

Let's first start with by inspecting our `reviews` table to see what data we have available to us.

In [ ]:
%sql SELECT * FROM reviews LIMIT 5;

## 2.2 Stopwords Removal

Our first stop is to apply stop-word removal - this involves identifying and removing words which commonly appear in English documents.

The purpose for this removal step is to help remove noise from our dataset - in turn helping our machine learning algorithms to better learn the correct representations within the data inputs.

However - we will see from our reviews example that just because it's a standard process doesn't always mean that it's the right solution for every problem!


### 2.2.1 Identifying Stopwords

Normally in a Python based workflow - stop-word removal is accomplished using standard NLP library functions, however we don't have this luxury in most SQL databases including `DuckDB`.

Luckily we can use our trusty `REGEXP_REPLACE` function to easily remove this huge list of English stop-words from our text data - these stop-words below are from the standard Python NLP library called `nltk`:

```text
a             about         above         after         again         ain  
all           am            an            and           any           are  
aren't        as            at            be            because       been  
before        being         below         between       both          but  
by            can           couldn        couldn't      d             did  
didn't        didn          do            does          doesn't       doesn  
doing         don't         don           down          during        each  
few           for           from          further       had           hadn't  
hadn          has           hasn't        hasn          have          haven't  
haven         having        he            her           here          hers  
herself       him           himself       his           how           i  
if            in            into          is            isn't         isn  
it            it's          its           itself        let's         ll  
m             ma            me            mightn        mightn't      more  
most          mustn         mustn't       my            myself        needn  
needn't       no            nor           not           now           o  
of            off           on            once          only          or  
other         our           ours          ourselves     out           over  
own           re            s             same          shan't        shan  
she           she's         should        should've     shouldn't     shouldn  
so            some          such          t             than          that  
that'll       the           their         theirs        them          themselves  
then          there         these         they          this          those  
through       to            too           under         until         up  
very          was           wasn't        wasn          we            were  
weren't       weren         what          when          where         which  
while         who           whom          why           will          with  
won't         won           wouldn        wouldn't      y             you  
you'd         you'll        you're        you've        your          yours  
yourself      yourselves
```

### 2.2.1 Implementing Stopword Removal

We can implement our stopwords removal this using a simple `REGEXP_REPLACE` function and a very, very, very, very long `OR` conditional expression.

Luckily for us - we can generate this regular expression using our favourite AI helper of choice, in fact - I used ChatGPT for mine!

Once the replacement is done - it's also common practice to "collapse" our final transformed text string by replacing multiple whitespace characters with a single one - this is due to the removal of words using word boundaries within the regular expression. We have seen a similar example in the `locations` HTML data that we manipulated in an earlier exercise.

We will also use the global case-insensitive match condition in our `REGEXP_REPLACE` - this makes sure we capture all of the occurences of these stopwords and we won't need to worry about case sensitivity. This option is important as we want all stop words to be removed regardless of whether they are capitalised or not.

Finally we apply a `LOWER` transformation on our `text_data` column to prepare ourselves for the next NLP analysis steps.

Some traditional NLP techniques perform better with all lowercase text vs mixed case due to lower chance of sparsity (uncommon values) - however some modern NLP techniques like named entity recognition, Q&A parsing and or any application that uses transfomers such as some embedding models like BERT.

**To summarize our tasks in the following SQL script**

1. Remove stopwords using `REGEXP_REPLACE`
2. Collapse multiple whitespaces into a single space
3. Lowercase the reviews text
4. Apply a trim to remove leading/trailing whitespace

Note some of the comments in the below SQL cell for more details - we'll be storing our outputs in a Pandas DataFrame variable called `reviews_stop_words_removed_df`

In [ ]:
%%sql reviews_stop_words_removed_df <<
SELECT
    review_id,
    # Keep these columns for analysis later!
    product_id,
    sentiment,
    star_rating,
    # Keep the original review text so we can compare
    review_text,
    # Apply lowercase
    TRIM(LOWER(
        REGEXP_REPLACE(
            # Remove stop-words
            REGEXP_REPLACE(
                review_text,
                '\b(i|me|my|myself|we|our|ours|ourselves|you|you''re|you''ve|you''ll|you''d|your|yours|yourself|yourselves|he|him|his|himself|she|she''s|her|hers|herself|it|it''s|its|itself|they|them|their|theirs|themselves|what|which|who|whom|this|that|that''ll|these|those|am|is|are|was|were|be|been|being|have|has|had|having|do|does|did|doing|a|an|the|and|but|if|or|because|as|until|while|of|at|by|for|with|about|against|between|into|through|during|before|after|above|below|to|from|up|down|in|out|on|off|over|under|again|further|then|once|here|there|when|where|why|how|all|any|both|each|few|more|most|other|some|such|no|nor|not|only|own|same|so|than|too|very|s|t|can|will|just|don|don''t|should|should''ve|now|d|ll|m|o|re|ve|y|ain|aren|aren''t|couldn|couldn''t|didn|didn''t|doesn|doesn''t|hadn|hadn''t|hasn|hasn''t|haven|haven''t|isn|isn''t|ma|mightn|mightn''t|mustn|mustn''t|needn|needn''t|shan|shan''t|shouldn|shouldn''t|wasn|wasn''t|weren|weren''t|won|won''t|wouldn|wouldn''t)\b',
                '',
                'gi'
            ),
        # Collapse multiple spaces into a single space
        '[\s]+', ' ', 'g'
        )
    )) AS transformed_review_text
FROM reviews;

### 2.2.2 Critically Analyze Reviews

Let's now compare the first row with the original and the updated text values to see the stop words removal in action and assess whether it's going to help us with our NLP problem.

It's very important to note that while removing stopwords is "technically" the correct way to deal with NLP text data - we still need to take a look carefully at our transformations to see if it's actually doing what we're intending!

In [ ]:
# Print out the first row
%sql SELECT * FROM reviews_stop_words_removed_df LIMIT 1;

In [ ]:
# Show the original review_text field
wrap_print(reviews_stop_words_removed_df.loc[0, "review_text"])

In [ ]:
# Show the transformed_review_text field
wrap_print(reviews_stop_words_removed_df.loc[0, "transformed_review_text"])

In our case for this single review - we can clearly see that our stop words have been removed and all our terms are now in lowercase - just like we expected...

However we can also see that removal of certain stop words totally alters the overall meaning of our review!

This is one simple example to demonstrate the importance of checking our data every step of the way - especially when following a "standardized" pipeline for NLP machine learning transformations.

For our following transformations - we'll actually avoid the stopword removal and process our reviews text as-is!

## 2.3 Simple Text Transformations

Since we understand now that removing stopwords might actually impact the meaning we can extract from each review - let's still apply some light transformations which will help our NLP techniques.

We'll apply the following steps:

1. Apply lowercase to all text
2. Remove any punctuation (using `REGEXP_REPLACE`)
3. Collapse any multiple whitespace characters

Let's store our outputs from these transformations in a variable called `transformed_reviews_df` and keep our original columns and add on a `transformed_review_text` field with our changes.

In [ ]:
%%sql transformed_reviews_df <<
SELECT
  review_id,
  product_id,
  sentiment,
  star_rating,
  review_text,
  LOWER(
    REGEXP_REPLACE(
      # Remove punctuation using character class
      REGEXP_REPLACE(review_text, '[\(\)&,.:;\\—!]', '', 'g'),
      # Collapse whitespace
      '[ ]+', ' ', 'g'
    )
  ) AS transformed_review_text
FROM reviews;

In [ ]:
transformed_reviews_df.head()

## 2.4 Bag of Words Analysis

Let's continue our exercise by implementing a simple bag-of-words transformation on our `transformed_reviews_df` - but first let's step through a single review to understand how we might apply our SQL queries.


### 2.4.1 Analyze Single Review

We begin our analysis by looking at an individual review `review_id = '000319b1'`

We'll need to use our `.loc` notation and our custom `wrap_print` function to print out both our original and transformed review text for inspection.

In [ ]:
%sql single_review_df << SELECT * FROM transformed_reviews_df WHERE review_id = '000319b1';

In [ ]:
single_review_df.head()

In [ ]:
# print out original review text
wrap_print(single_review_df.loc[0, "review_text"])

In [ ]:
# print out transformed review text
wrap_print(single_review_df.loc[0, "transformed_review_text"])

### 2.4.2 Implement Tokenization

Here's where we start diving into the deep- we'll use some relatively advanced SQL techniques here to convert our text string into a tokenized representation of our reviews data.

These techniques really come in handy when dealing with complex data structures within SQL and it was a true threshold concept for me when I was learning in my career!

The steps we'll use are as follows:

1. Use `REGEXP_SPLIT_TO_ARRAY` to convert our text string into an array of "terms"
2. Add on an array from 1 to the `ARRAY_LENGTH` of our document to track "position"
3. Slice our original document array of terms using our "position"

There's a lot of techniques in the following SQL query so I've tried my best to add comments to make things as clear as possible - please feel free to break up the CTE into multiple steps to understand each transformation step-by-step if you need!

Let's also store our outputs as `single_review_tokenized_df`


In [ ]:
%%sql single_review_tokenized_df <<
WITH cte_arrays AS (
SELECT
    # 1. Convert our text document into an array
    REGEXP_SPLIT_TO_ARRAY(transformed_review_text, '\s+') AS term_array
FROM single_review_df
),
cte_range AS (
SELECT
    term_array,
    # In DuckDB specifically we need to use the .unnest to get the raw value!
    i.unnest AS position
FROM cte_arrays
# 2. Apply CROSS JOIN UNNEST on our array to get the "position" [1, 2, 3 ... N]
# Note: we need the +1 here to account for 0 based array indexing with RANGE
CROSS JOIN UNNEST(RANGE(1, ARRAY_LENGTH(term_array) + 1)) AS i
)
SELECT
    # Slice out the original term array of our document using the position
    term_array[position] AS term,
    position
FROM cte_range
ORDER BY position

In [ ]:
single_review_tokenized_df

### 2.4.3 Implement Bag-of-Words

At this stage - we can simply perform a `GROUP BY COUNT` on our output DataFrame `single_review_tokenized_df` to generate the required output for our Bag of Words analysis.

Let's keep our outputs as `single_review_bag_of_words_df`

In [ ]:
%%sql single_review_bag_of_words_df <<
SELECT
  term,
  COUNT(*) AS frequency
FROM single_review_tokenized_df
GROUP BY term
ORDER BY term;

In [ ]:
single_review_bag_of_words_df

## 2.5 Tokenize All Reviews

We can now remove our `WHERE` filter and apply these same tokenization steps we covered on our entire collection of customer reviews.

We'll just need to make sure we keep the `review_id` in our query to make sure we keep all of the terms that are related to each document!


## 2.5.1 SQL Implementation

Let's try this with a very similar SQL query to our previous step but this time we'll use our complete reviews dataset `transformed_reviews_df`.

Let's also store our outputs as `reviews_tokenized_df`


In [ ]:
%%sql reviews_tokenized_df <<
WITH cte_arrays AS (
SELECT
    review_id,
    # 1. Convert our text document into an array
    REGEXP_SPLIT_TO_ARRAY(transformed_review_text, '\s+') AS term_array
FROM transformed_reviews_df
),
cte_range AS (
SELECT
    review_id,
    term_array,
    # In DuckDB specifically we need to use the .unnest to get the raw value!
    i.unnest AS position
FROM cte_arrays
# 2. Apply CROSS JOIN UNNEST on our array to get the "position" [1, 2, 3 ... N]
# Note: we need the +1 here to account for 0 based array indexing with RANGE
CROSS JOIN UNNEST(RANGE(1, ARRAY_LENGTH(term_array) + 1)) AS i
)
SELECT
    review_id,
    # Slice out the original term array of our document using the position
    term_array[position] AS term,
    position
FROM cte_range
ORDER BY review_id, position

In [ ]:
reviews_tokenized_df

## 2.6 Bag-of-Words Analysis

Let's now extend our Bag-of-Words analysis to the entire range of reviews. For now we will stick with uni-gram terms 1-at-a-time so we can see the logic in action!

### 2.6.1 Generating Vocabulary

Our first challenge for our bag-of-words analysis is to generate what's known as a "vocabulary" of all possible terms/tokens which appear in our collection of customer reviews (our corpus).

We can simply take a `DISTINCT` of our `term` column in our `reviews_tokenized_df` DataFrame to get our initial uni-gram vocabulary one at a time!

In [ ]:
%sql SELECT DISTINCT term FROM reviews_tokenized_df ORDER BY term;

### 2.6.2 Generating Term Frequency & BoW

We can now use this same vocabulary logic to generate our final output for the Bag-of-Words uni-gram analysis.

We'll need to create a CTE with our previous logic and join onto this to create a simple count of each term within each document. This is also known as the `term frequency`

**One key step** here is that we will need to create a complete combination of reviews and terms - before performing a `LEFT JOIN` on our reviews terms using this combination as our "base" dataset that is kept on the left side of our join.

This ensures that we do not lose any vocabulary terms as we perform the join onto our term frequency counts for each customer review!

Let's store our output into the new variable `reviews_unigram_bag_of_words_df`

In [ ]:
%%sql reviews_unigram_bag_of_words_df <<
WITH cte_vocabulary AS (
  SELECT DISTINCT
    term
  FROM reviews_tokenized_df
),
cte_reviews AS (
  SELECT DISTINCT
    review_id
  FROM reviews_tokenized_df
),
# Combine all unique reviews and vocabulary to form our base
cte_all_terms_and_reviews_combo AS (
  SELECT
    review_id,
    term
  FROM cte_reviews
  CROSS JOIN cte_vocabulary
),
# Technically we calculate the term freq as part of our bag of words analysis!
cte_term_frequency AS (
  SELECT
    review_id,
    term,
    COUNT(*) AS frequency
  FROM reviews_tokenized_df
  GROUP BY review_id, term
)
# Final bag of words output!
SELECT
  # we need to use our combo for these columns
  combo.review_id,
  combo.term,
  # we use coalesce here to replace missing values with 0
  COALESCE(tf.frequency, 0) AS term_frequency
FROM cte_all_terms_and_reviews_combo AS combo
LEFT JOIN cte_term_frequency AS tf
  ON combo.review_id = tf.review_id
  AND combo.term = tf.term
# Apply a simple order by review_id and term in alphabetical order
ORDER BY combo.review_id, combo.term

In [ ]:
# Let's just take a look at our very first review_id we were inspecting before
%sql SELECT * FROM reviews_unigram_bag_of_words_df WHERE review_id = '000319b1';

## 2.7 Document Frequency

Let's use our exact same uni-gram terms to implement `document frequency` which is the count of unique documents or reviews where a specific term appears.

We'll need to perform a similar calculation as our `term frequency` but this time instead of using a `GROUP BY COUNT` within each review or document - we will instead apply a `COUNT DISTINCT` on the `review_id` for each uni-gram term.

Let's store our output as `reviews_unigram_document_frequency_df`


In [ ]:
%%sql reviews_unigram_document_frequency_df <<
WITH cte_document_frequency AS (
SELECT
    term,
    COUNT(DISTINCT review_id) AS document_frequency
FROM reviews_tokenized_df
GROUP BY term
),
cte_vocab AS (
SELECT DISTINCT
    term
FROM reviews_tokenized_df
)
# final output
SELECT
    vocab.term,
    COALESCE(df.document_frequency, 0) AS document_frequency,
FROM cte_vocab AS vocab
LEFT JOIN cte_document_frequency AS df
    ON vocab.term = df.term
ORDER BY document_frequency DESC;

Let's inspect our `reviews_unigram_document_frequency_df` output DataFrame to see if there are any insights we can extract.

Maybe we can take a look at the top 25 terms by document frequency first!

Some of these terms might be perfect to apply as stopwords as opposed to our standard list of stopwords from `nltk`

In [ ]:
reviews_unigram_document_frequency_df[:25]

## 2.8 TF-IDF Calculation

Now let's put together our term frequency and document frequency to generate one of the most common NLP metrics called `TF-IDF` or term frequency inverse document frequency.

This helps us generate a metric for each term within each review so we can evaluate the following:

1. How common is this term in the current review?
2. How rare is this term across all reviews?

We'll also need to apply a natural log transformation to our value to scale our final output. This helps us keep our final `TD-IDF` values in the same range of values - however in practice, the ranking of these values are more important than the actual scores!

### 2.8.1 Uni-gram TF-IDF

Let's first try this TF-IDF calculation using our uni-gram terms!

We can store our outputs for this transformation as `reviews_unigram_tfidf_df`


In [ ]:
%%sql reviews_unigram_tfidf_df <<
# calculate the term frequency
WITH cte_term_frequency AS (
  SELECT
    review_id,
    term,
    COUNT(*) AS term_frequency
  FROM reviews_tokenized_df
  GROUP BY review_id, term
),
cte_document_frequency AS (
  SELECT
    term,
    COUNT(DISTINCT review_id) AS document_frequency
  FROM reviews_tokenized_df
  GROUP BY term
),
cte_total_document_count AS (
  SELECT
    COUNT(DISTINCT review_id) AS document_count
  FROM reviews_tokenized_df
),
cte_combined AS (
  # Combine both CTEs to get TF-IDF
  SELECT
    # we need to use our combo for these columns
    tf.review_id,
    tf.term,
    # we use coalesce here to replace missing values with 0
    COALESCE(tf.term_frequency, 0) AS term_frequency,
    COALESCE(df.document_frequency, 0) AS document_frequency,
    # We use this document count to scale out TF-IDF calculation
    docs.document_count
  FROM cte_term_frequency AS tf
  LEFT JOIN cte_document_frequency AS df
    ON tf.term = df.term
  CROSS JOIN cte_total_document_count AS docs
)
# Final output
SELECT
  review_id,
  term,
  term_frequency,
  document_frequency,
  term_frequency * LN(document_count / document_frequency) AS tf_idf
FROM cte_combined
# Let's order by descending tf_idf within each review ID
ORDER BY review_id, tf_idf DESC

In [ ]:
# Let's just take a look at our very first review_id we were inspecting before
%sql SELECT * FROM reviews_unigram_tfidf_df WHERE review_id = '000319b1';

### 2.8.2 Bi-gram TF-IDF

Sometimes looking at terms/words one-at-a-time only gives us so much information.

We might inadvertantly reduce specific terms such as `Los Angeles` or `not great` into their individual uni-gram parts which lowers the contextual information we get from our text data.

Let's introduce our `LAG` window function to help us combine our terms based off their positions before we get busy applying our same transformations to calculate `TF-IDF`

We can store outputs as `reviews_bigram_tfidf_df`

In [ ]:
%%sql reviews_bigram_tfidf_df <<
WITH cte_bigrams AS (
SELECT
    review_id,
    # Use LAG window function to take combine 2 positional terms into a bi-gram
    LAG(term) OVER (PARTITION BY review_id ORDER BY position) || ' ' || term AS term
FROM reviews_tokenized_df
),
# calculate the term frequency using our cte_bigrams table
cte_term_frequency AS (
  SELECT
    review_id,
    term,
    COUNT(*) AS term_frequency
  FROM cte_bigrams
  # When we use our window function we also have the last trailing term as NULL
  WHERE term IS NOT NULL
  GROUP BY review_id, term
),
# calculate the document frequency using our cte_bigrams table
cte_document_frequency AS (
  SELECT
    term,
    COUNT(DISTINCT review_id) AS document_frequency
  FROM cte_bigrams
  # When we use our window function we also have the last trailing term as NULL
  WHERE term IS NOT NULL
  GROUP BY term
),
cte_total_document_count AS (
  SELECT
    COUNT(DISTINCT review_id) AS document_count
  FROM cte_bigrams
),
cte_combined AS (
  # Combine both CTEs to get TF-IDF
  SELECT
    # we need to use our combo for these columns
    tf.review_id,
    tf.term,
    # we use coalesce here to replace missing values with 0
    COALESCE(tf.term_frequency, 0) AS term_frequency,
    COALESCE(df.document_frequency, 0) AS document_frequency,
    # We use this document count to scale out TF-IDF calculation
    docs.document_count
  FROM cte_term_frequency AS tf
  LEFT JOIN cte_document_frequency AS df
    ON tf.term = df.term
  CROSS JOIN cte_total_document_count AS docs
)
# Final output
SELECT
  review_id,
  term,
  term_frequency,
  document_frequency,
  term_frequency * LN(document_count / document_frequency) AS tf_idf
FROM cte_combined
# Let's order by descending tf_idf within each review ID
ORDER BY review_id, tf_idf DESC

In [ ]:
# Let's just take a look at our very first review_id we were inspecting before
%sql SELECT * FROM reviews_bigram_tfidf_df WHERE review_id = '000319b1';

## 2.9 Combining TF-IDF and Products

Let's now use our bi-gram TF-IDF dataset `reviews_bigram_tfidf_df` to help us understand how each of our metrics: term frequency, document frequency and `TF-IDF` compare when we start to aggregate these to rank the top words for each `star_rating` for each `product`

We'll build upon our work previously and join in more information about our products!

### 2.9.1 Joining Multiple Tables

We'll need to join onto our `reviews` table to make sure we can split out the `star_rating` and also join onto our `products` table in order to get the `product_name` field.

Let's revisit our database entity relationship diagram below to reference as we script up our SQL code.

<iframe width="100%" height="600" src='https://dbdiagram.io/e/685279b3f039ec6d36c0c7e9/68527d19f039ec6d36c1813e'> </iframe>

### 2.9.2 Product Star Ratings

Let's create our joint table `product_star_rating_reviews_df` by averaging up our term frequency, document frequency and TF-IDF by each product and star_rating.

We'll also apply some ranking using the `ROW_NUMBER` window function to specify the top few terms for each product and star_rating accordingly.

The SQL below is a little bit crazy - but don't worry as this just helps us pull together all of our individual analyses into one single joint table.

Let's also store this as `product_star_rating_reviews_df`

In [ ]:
%%sql product_star_rating_reviews_df <<
WITH cte_base AS (
SELECT
    products.product_name,
    reviews.star_rating,
    tfidf.term,
    AVG(tfidf.term_frequency) AS term_frequency,
    AVG(tfidf.document_frequency) AS document_frequency,
    AVG(tfidf.tf_idf) AS tf_idf
FROM reviews_bigram_tfidf_df AS tfidf
INNER JOIN reviews
    ON tfidf.review_id = reviews.review_id
INNER JOIN products
    ON reviews.product_id = products.product_id
GROUP BY 1,2,3
),
cte_ranked AS (
SELECT
    product_name,
    star_rating,
    term,
    ROW_NUMBER() OVER (PARTITION BY product_name, star_rating ORDER BY term_frequency DESC) AS tf_rank,
    ROW_NUMBER() OVER (PARTITION BY product_name, star_rating ORDER BY document_frequency DESC) AS df_rank,
    ROW_NUMBER() OVER (PARTITION BY product_name, star_rating ORDER BY tf_idf DESC) AS tfidf_rank
FROM cte_base
)
# now combine everything into one table using a big self join!
SELECT
    tf.product_name,
    tf.star_rating,
    tf.tf_rank AS term_ranking,
    tf.term AS tf_term,
    df.term AS df_term,
    tfidf.term AS tfidf_term
FROM cte_ranked AS tf
INNER JOIN cte_ranked AS df
    ON tf.product_name = df.product_name
    AND tf.star_rating = df.star_rating
    AND tf.tf_rank = df.df_rank
INNER JOIN cte_ranked AS tfidf
    ON tf.product_name = tfidf.product_name
    AND tf.star_rating = tfidf.star_rating
    AND tf.tf_rank = tfidf.tfidf_rank
ORDER BY 1,2,3;

### 2.9.3 California Classic Reviews

Let's take a look at a single product and we'll pull out only 1 and 5 star ratings and compare the top 10 terms by each of our metrics!

In [ ]:
%%sql
SELECT *
FROM product_star_rating_reviews_df
WHERE product_name = 'California Classics'
AND star_rating IN (1, 5)
AND term_ranking <= 10;

### 2.9.4 Analyzing California Classics Reviews

Here we can see that the `term frequency` and `document frequency` terms seem to line up with what we are expecting.

Negative ratings seem to state a few things that weren't liked in the term frequency - while the document frequency paints a very clear picture of negative sentiment.


However - when we look at the `TF-IDF` terms - we can notice that these are actually the same!

These terms although they are high scoring in TF-IDF across the entire corpus or collection of documents. Maybe we don't have enough information when it comes to discriminating just within individual product reviews.

In our example - it looks like all relevant top terms are unique to this California Classics tour product but when we zoom into our product - the TF-IDF top terms also come short in regards to insightful analysis.

## 2.10 Product Specific TF-IDF Metrics

Let's now extend our analysis even further and we will group together only our documents for `California Classics` product reviews.

This should solve the problem we were facing when identifying relevant TF-IDF terms for one specific product!

This query below is going to a little long-winded but it's just a combination of a few steps that we've covered above all put into a single long series of CTEs. The only difference is that we'll need to join onto our `reviews` and `products` table early on to filter out only product reviews for our California Classics tour product.

In [ ]:
%%sql cali_classics_analysis_df <<
WITH cte_bigrams AS (
SELECT
    tokens.review_id,
    # Use LAG window function to take combine 2 positional terms into a bi-gram
    LAG(tokens.term) OVER (PARTITION BY tokens.review_id ORDER BY tokens.position) || ' ' || term AS term
FROM reviews_tokenized_df AS tokens
INNER JOIN reviews
    ON tokens.review_id = reviews.review_id
INNER JOIN products
    ON reviews.product_id = products.product_id
WHERE products.product_name = 'California Classics'
),
cte_tf AS (
SELECT
    review_id,
    term,
    COUNT(*) AS term_frequency
FROM cte_bigrams
# Apply a WHERE filter to remove the first bi-gram which will be NULL
WHERE term IS NOT NULL
GROUP BY
    review_id,
    term
),
cte_df AS (
SELECT
    term,
    COUNT(DISTINCT review_id) AS document_frequency
FROM cte_bigrams
# Apply a WHERE filter to remove the first bi-gram which will be NULL
WHERE term IS NOT NULL
GROUP BY
    term
),
cte_doc_count AS (
SELECT COUNT(DISTINCT review_id) AS document_count FROM cte_bigrams
),
cte_vocab AS (
SELECT DISTINCT
    term
FROM cte_df
),
cte_combined AS (
SELECT
    tf.review_id,
    vocab.term,
    COALESCE(tf.term_frequency, 0) AS term_frequency,
    COALESCE(df.document_frequency, 0) AS document_frequency,
    docs.document_count
FROM cte_vocab AS vocab
LEFT JOIN cte_tf AS tf
    ON vocab.term = tf.term
LEFT JOIN cte_df AS df
    ON vocab.term = df.term
CROSS JOIN cte_doc_count AS docs
),
cte_nlp_metrics AS (
SELECT
    nlp.review_id,
    reviews.star_rating,
    nlp.term,
    nlp.term_frequency,
    nlp.document_frequency,
    nlp.term_frequency * LN(nlp.document_count / nlp.document_frequency) AS tf_idf
FROM cte_combined AS nlp
INNER JOIN reviews
    ON nlp.review_id = reviews.review_id
),
cte_rating_metrics AS (
SELECT
    star_rating,
    term,
    AVG(term_frequency) AS term_frequency,
    AVG(document_frequency) AS document_frequency,
    AVG(tf_idf) AS tf_idf
FROM cte_nlp_metrics AS tfidf
GROUP BY
    star_rating,
    term
),
cte_ranked AS (
SELECT
    star_rating,
    term,
    ROW_NUMBER() OVER (PARTITION BY star_rating ORDER BY term_frequency DESC) AS tf_rank,
    ROW_NUMBER() OVER (PARTITION BY star_rating ORDER BY document_frequency DESC) AS df_rank,
    ROW_NUMBER() OVER (PARTITION BY star_rating ORDER BY tf_idf DESC) AS tfidf_rank
FROM cte_rating_metrics
)
# now combine everything into one table
SELECT
    tf.star_rating,
    tf.tf_rank AS term_ranking,
    tf.term AS tf_term,
    df.term AS df_term,
    tfidf.term AS tfidf_term
FROM cte_ranked AS tf
INNER JOIN cte_ranked AS df
    ON tf.star_rating = df.star_rating
    AND tf.tf_rank = df.df_rank
INNER JOIN cte_ranked AS tfidf
    ON tf.star_rating = tfidf.star_rating
    AND tf.tf_rank = tfidf.tfidf_rank
ORDER BY 1,2;


### 2.10.1 Inspecting Outputs

Now that this analysis is complete - let's see if our TF-IDF terms give us more insights around the 1 and 5 star ratings!

In [ ]:
%%sql
SELECT *
FROM cali_classics_analysis_df
WHERE star_rating IN (1, 5)
AND term_ranking <= 10;

### 2.10.2 Critically Thinking About TF-IDF

So in theory - although TF-IDF is great at highlighting distinctive or unique terms within a document relative to the overall collection of documents in a corpus - it's not always that simple!

In our example - we can see that creating a sub-corpus of documents that are relevant to each of our tour products yields more insights out of our TD-IDF analysis. However - we can also see that the TF-IDF when we reduce the scope to only look at an individual product also yields quite similar results to the overall term-frequency metrics that we calculate.

This is another timely reminder that we always need to be quite critical when analyzing our data and to always be open to experimenting with a variety of methods and approaches!

## 2.11 Top Terms for 5 Star Reviews

Let's now implement a final SQL script for this NLP text cleaning tutorial where we can perform the in-product TF-IDF analysis for our reviews to return top 10 terms for each product!

We'll re-use much of our logic from previous steps - but this time we'll just remove our `WHERE` filter on the product and apply it instead on the `star_rating` field.

Let's store our final output as `reviews_5_star_terms_df`

In [ ]:
%%sql reviews_5_star_terms_df <<
WITH cte_terms AS (
SELECT
    products.product_name,
    tokens.review_id,
    # Use LAG window function to take combine 2 positional terms into a bi-gram
    LAG(tokens.term) OVER (PARTITION BY tokens.review_id ORDER BY tokens.position) || ' ' || term AS term
FROM reviews_tokenized_df AS tokens
INNER JOIN reviews
    ON tokens.review_id = reviews.review_id
INNER JOIN products
    ON reviews.product_id = products.product_id
WHERE reviews.star_rating = 5
),
cte_tf AS (
SELECT
    product_name,
    review_id,
    term,
    COUNT(*) AS term_frequency
FROM cte_terms
# Apply a WHERE filter to remove the first bi-gram which will be NULL
WHERE term IS NOT NULL
GROUP BY
    product_name,
    review_id,
    term
),
cte_df AS (
SELECT
    product_name,
    term,
    COUNT(DISTINCT review_id) AS document_frequency
FROM cte_terms
# Apply a WHERE filter to remove the first bi-gram which will be NULL
WHERE term IS NOT NULL
GROUP BY
    product_name,
    term
),
cte_doc_count AS (
SELECT
    product_name,
    COUNT(DISTINCT review_id) AS document_count
FROM cte_terms
GROUP BY product_name
),
cte_vocab AS (
SELECT DISTINCT
    product_name,
    term
FROM cte_df
),
cte_combined AS (
SELECT
    tf.product_name,
    tf.review_id,
    vocab.term,
    COALESCE(tf.term_frequency, 0) AS term_frequency,
    COALESCE(df.document_frequency, 0) AS document_frequency,
    docs.document_count
FROM cte_vocab AS vocab
LEFT JOIN cte_tf AS tf
    ON vocab.product_name = tf.product_name
    AND vocab.term = tf.term
LEFT JOIN cte_df AS df
    ON vocab.product_name = df.product_name
    AND vocab.term = df.term
LEFT JOIN cte_doc_count AS docs
    ON docs.product_name = tf.product_name
),
cte_nlp_metrics AS (
SELECT
    nlp.product_name,
    nlp.review_id,
    reviews.star_rating,
    nlp.term,
    nlp.term_frequency,
    nlp.document_frequency,
    nlp.term_frequency * LN(nlp.document_count / nlp.document_frequency) AS tf_idf
FROM cte_combined AS nlp
INNER JOIN reviews
    ON nlp.review_id = reviews.review_id
),
cte_rating_metrics AS (
SELECT
    product_name,
    star_rating,
    term,
    AVG(term_frequency) AS term_frequency,
    AVG(document_frequency) AS document_frequency,
    AVG(tf_idf) AS tf_idf
FROM cte_nlp_metrics AS tfidf
GROUP BY
    product_name,
    star_rating,
    term
),
cte_ranked AS (
SELECT
    product_name,
    star_rating,
    term,
    ROW_NUMBER() OVER (PARTITION BY product_name, star_rating ORDER BY term_frequency DESC) AS tf_rank,
    ROW_NUMBER() OVER (PARTITION BY product_name, star_rating ORDER BY document_frequency DESC) AS df_rank,
    ROW_NUMBER() OVER (PARTITION BY product_name, star_rating ORDER BY tf_idf DESC) AS tfidf_rank
FROM cte_rating_metrics
)
# now combine everything into one table
SELECT
    tf.product_name,
    tf.star_rating,
    tf.tf_rank AS term_ranking,
    tf.term AS tf_term,
    df.term AS df_term,
    tfidf.term AS tfidf_term
FROM cte_ranked AS tf
INNER JOIN cte_ranked AS df
    ON tf.product_name = df.product_name
    AND tf.star_rating = df.star_rating
    AND tf.tf_rank = df.df_rank
INNER JOIN cte_ranked AS tfidf
    ON tf.product_name = tfidf.product_name
    AND tf.star_rating = tfidf.star_rating
    AND tf.tf_rank = tfidf.tfidf_rank
ORDER BY 1,2,3;


In [ ]:
# Set up Pandas option to help us show all rows from our output
pd.set_option('display.max_rows', None)

%sql SELECT * FROM reviews_5_star_terms_df WHERE term_ranking <= 10;

In [ ]:
# Reset Pandas option to only show top 10 rows
pd.set_option('display.max_rows', 10)

# 3. Implement A/B Framework

The final part of our NLP challenge is to design a measurement framework to quantify the uplift of an AI experiment.

Although we won't be able to implement the advanced NLP components ourselves using SQL - we will definitely be able to analyze the sales uplift by investigating our `visits`, `sales` and `features` tables in our SQL database.

In our Explore California business example - I've generated some simulated AI experiment results for the year 2026.

In the first 3 months of the calendar year - 50% of the website visitors are exposed to an AI powered "NLP search" tool.

We will be using this data to compare our sales performance for this 3 month period compared to a few different baselines.

Let's again revisit our ERD to make sure we can visualize how we will join our tables together to come up with a technical solution.

<iframe width="100%" height="600" src='https://dbdiagram.io/e/685279b3f039ec6d36c0c7e9/68527d19f039ec6d36c1813e'> </iframe>

## 3.1 Inspect Raw Data

We can begin by taking a look at the `sales`, `visits` and `features` tables - these will be key to our analysis!

We can also link up our sales with each relevant product to identify the price in $USD using the `product_id` - we'll keep this up our sleeve for later!

In [ ]:
%%sql
SELECT * FROM sales LIMIT 5;

We can see that the `visit_id` can be used to link each sales transaction to specific website visit - let's take a look at what we have in the visits table.

In [ ]:
%%sql
SELECT * FROM visits LIMIT 5;

And finally - we've got a `features` table which identifies the `visit_id` values where an AI powered feature was shown.

For our first NLP search feature - we can apply a filter on this table for "Search"

In [ ]:
%%sql
SELECT * FROM features
WHERE feature = 'Search'
LIMIT 5;

## 3.3 Experimental Analysis

Let's have a go at combining all of these tables to generate a single table where we can apply all of our analysis.

The required columns that we'll need for this table are below:

* visit_timestamp
* visit_id
* user_id
* feature_flag (if it exists)
* sale_flag (if it exists)
* sale_amount (use the `products` table to find the USD price)

Let's also filter this table to only include the first 3 months of 2026 - this is when our theoretical AI experiment is running.

We will store out outputs as `experiment_analysis_df`

In [ ]:
%%sql experiment_analysis_df <<
SELECT
    visits.visit_timestamp,
    visits.visit_id,
    visits.user_id,
    CASE WHEN features.feature IS NOT NULL THEN 1 ELSE 0 END AS feature_flag,
    CASE WHEN sales.sale_id IS NOT NULL THEN 1 ELSE 0 END AS sale_flag,
    COALESCE(products.price_usd, 0) AS sale_amount
FROM visits
LEFT JOIN features
    ON visits.visit_id = features.visit_id
LEFT JOIN sales
    ON visits.visit_id = sales.visit_id
LEFT JOIN products
    ON sales.product_id = products.product_id
WHERE visits.visit_timestamp BETWEEN DATE '2026-01-01' AND DATE '2026-03-31';

In [ ]:
experiment_analysis_df.head()

## 3.4 Answering Business Questions

Let's use `experiment_analysis_df` to answer a few relevant questions at a macro level before we zoom into the experiment analysis.

1. How many sales are there per month and what is the monthly total sale amount?
2. What is the total number of visits per month?
3. What percentage of visits lead to a sale?

### 3.4.1 Question 1

In [ ]:
%%sql
# How many sales are there per month and what is the monthly total sale amount?
SELECT
    DATE_TRUNC('MON', visit_timestamp) AS sales_month,
    SUM(sale_flag) AS sales_count,
    SUM(sale_amount) AS sales_amount
FROM experiment_analysis_df
WHERE sale_flag = 1
GROUP BY 1
ORDER BY 1;

### 3.4.2 Question 2

In [ ]:
%%sql
# 2. What is the total number of visits per month?
SELECT
    DATE_TRUNC('MON', visit_timestamp) AS sales_month,
    COUNT(DISTINCT visit_id) AS visit_count
FROM experiment_analysis_df
GROUP BY 1
ORDER BY 1;

### 3.4.3 Question 3

In [ ]:
%%sql
# What percentage of visits lead to a sale?
SELECT
    DATE_TRUNC('MON', visit_timestamp) AS sales_month,
    COUNT(DISTINCT visit_id) AS visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS sales_count,
    sales_count / visit_count AS conversion_rate
FROM experiment_analysis_df
GROUP BY 1
ORDER BY 1;

## 3.5 Experiment Questions

Now let's add back in the extra slice of the `feature_flag` to analyze our sales and conversion performance in a single table.

In [ ]:
%%sql
SELECT
    DATE_TRUNC('MON', visit_timestamp) AS sales_month,
    feature_flag,
    COUNT(DISTINCT visit_id) AS visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS sales_count,
    SUM(sale_amount) AS sales_amount,
    sales_count / visit_count AS conversion_rate
FROM experiment_analysis_df
GROUP BY 1,2
ORDER BY 1,2;

## 3.6 Quantifying Uplift

Here we can see there is some slight uplift for the visits where the `feature_flag` is present. Let's quantify the exact aggregated uplift for the entire experiment period.

There are many many ways to do this using SQL - but let's keep this simple and split up our analysis into 2 CTEs and combine them together to calculate the uplift metrics.

We generally will refer to the existing experience or the records where `feature_flag = 0` as the "control" and the group which were exposed to the feature would be considered as the "target" group.

Because we can see that traffic is roughly split to 50/50 for this experiment - we can calculate incremental values by simply finding the difference between the target and control sale counts and amounts values.

We'll store our output as `experiment_results_df`

In [ ]:
%%sql experiment_results_df <<
WITH cte_control AS (
SELECT
    COUNT(DISTINCT visit_id) AS control_visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS control_sales_count,
    SUM(sale_amount) AS control_sales_amount,
    control_sales_count / control_visit_count AS control_conversion_rate
FROM experiment_analysis_df
WHERE feature_flag = 0
),
cte_target AS (
SELECT
    COUNT(DISTINCT visit_id) AS target_visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS target_sales_count,
    SUM(sale_amount) AS target_sales_amount,
    target_sales_count / target_visit_count AS target_conversion_rate
FROM experiment_analysis_df
WHERE feature_flag = 1
)
SELECT
    # uplift metrics
    target_sales_count - control_sales_count AS incremental_sales_count,
    target_sales_amount - control_sales_amount AS incremental_sales_amount,
    100 * ( target_sales_count / control_sales_count - 1 ) AS sales_count_uplift,
    100 * ( target_sales_amount / control_sales_amount - 1 ) AS sales_amount_uplift,
    100 * ( target_conversion_rate / control_conversion_rate - 1 ) AS conversion_uplift,
    control.*,
    target.*
FROM cte_target AS target
CROSS JOIN cte_control AS control;

In [ ]:
experiment_results_df

## 3.7 Z-Test for Two Proportions

We can take this one step further and calculate the statistical validity of our experiment using what's known as an A/B test.

We'll use SQL to implement what is known as a "test of two proportions" where we will calculate a z-score using some inputs from our experiment analysis.

### 3.7.1 One-Tail vs Two-Tailed Tests

For our experiment - since we have an uplift in the conversion rate - we want to perform a one-tailed test of two proportions test as we are only interested in testing if this positive increase is in fact significant or not.

Generally these tests are performed at an alpha level = 0.05 allowing for a 5% error or false-positive rate from our given experiment results.

A one-tailed test is comparing one group to another when trying to confirm a hypothesis that our "target" group is performing better than our "control" group. A two-tailed test is just trying to confirm a different hypothesis - that our target group is different to our control where the directionality of the test is not considered.

### 3.7.2 Calculating the Z-Score

For our tests - we will want to set the z-score critical value threshold at 1.645 for a 5% false positive rate for our one-tailed test. The z-score a statistical measure that shows how unusual or typical a result is when compared to the null-hypothesis.

In our case - we want a higher z-score to disprove our null hypothesis that the target group does not perform better than the control group. A score near 0 means that the results from the experiment are likely to follow the null hypothesis - while a higher value provides us more information to decide whether the experiment results are statistically significant.

The z-score is calculated as below:

$z = \frac{p_1 - p_2}{\sqrt{p(1 - p) \left( \frac{1}{n_1} + \frac{1}{n_2} \right)}}$

Where:

- $p_1 = \dfrac{x_1}{n_1}$ — Target group conversion rate  
- $p_2 = \dfrac{x_2}{n_2}$ — Control group conversion rate
- $p = \dfrac{x_1 + x_2}{n_1 + n_2}$ — Overall conversion rate
- $x_1$, $x_2$ — number of sales in each target and control group  
- $n_1$, $n_2$ — number of visits/observations in each target and control group

### 3.7.3 Z-Score Implementation in SQL

We can implement this directly in SQL as we already have most of these values ready to go - the only thing we'll need to add in is the overall conversion rate by adding up our visits and sales for each group.

In [ ]:
%%sql
SELECT
  (target_sales_count + control_sales_count) /
    (target_visit_count + control_visit_count) AS overall_conversion_rate,
  # numerator
  (target_conversion_rate - control_conversion_rate) /
  # denominator
  SQRT(
    overall_conversion_rate * (1 - overall_conversion_rate) *
    (1 / target_visit_count + 1 / control_visit_count)
  ) AS z_score,
  CASE WHEN z_score >= 1.645 THEN 'Significant' ELSE 'Not Significant' END AS test_result
FROM experiment_results_df;

## 3.8 Sequential Comparison

Although the best practice is to compare sales and conversion for the exact same analysis period using some sort of split-test or A/B test. We should also explore a few other methods of measuring impact to illustrate the key differences and potential limitations of other baselines.


### 3.8.1 Comparing 3 Months Before

Let's say we want to compare just a simple 3 months before and 3 months during the experiment - let's just assume that we haven't split the traffic at all or we didn't capture the individual `feature` value for each visit but we just blindly ran a campaign.

Let's compare the 3 months from Jan to March 2026 with October to Dec 2025 - and we'll see that there is still an uplift!

In [ ]:
%%sql
WITH cte_base AS (
SELECT
    CASE
        WHEN visits.visit_timestamp BETWEEN DATE '2025-10-01' AND DATE '2025-12-31' THEN 'before'
        ELSE 'after'
    END AS experiment_group,
    visits.visit_id,
    visits.user_id,
    CASE WHEN sales.sale_id IS NOT NULL THEN 1 ELSE 0 END AS sale_flag,
    COALESCE(products.price_usd, 0) AS sale_amount
FROM visits
LEFT JOIN sales
    ON visits.visit_id = sales.visit_id
LEFT JOIN products
    ON sales.product_id = products.product_id
WHERE visits.visit_timestamp BETWEEN DATE '2025-10-01' AND DATE '2026-03-31'
),
cte_before AS (
SELECT
    COUNT(DISTINCT visit_id) AS before_visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS before_sales_count,
    SUM(sale_amount) AS before_sales_amount,
    before_sales_count / before_visit_count AS before_conversion_rate
FROM cte_base
WHERE experiment_group = 'before'
),
cte_after AS (
SELECT
    COUNT(DISTINCT visit_id) AS after_visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS after_sales_count,
    SUM(sale_amount) AS after_sales_amount,
    after_sales_count / after_visit_count AS after_conversion_rate
FROM cte_base
WHERE experiment_group = 'after'
),
cte_metrics AS (
SELECT
    # uplift metrics
    after_sales_count - before_sales_count AS incremental_sales_count,
    after_sales_amount - before_sales_amount AS incremental_sales_amount,
    100 * ( after_sales_count / before_sales_count - 1 ) AS sales_count_uplift,
    100 * ( after_sales_amount / before_sales_amount - 1 ) AS sales_amount_uplift,
    100 * ( after_conversion_rate / before_conversion_rate - 1 ) AS conversion_uplift,
    before.*,
    after.*
FROM cte_after AS after
CROSS JOIN cte_before AS before
)
SELECT
  (after_sales_count + before_sales_count) /
    (after_visit_count + before_visit_count) AS overall_conversion_rate,
  # numerator
  (after_conversion_rate - before_conversion_rate) /
  # denominator
  SQRT(
    overall_conversion_rate * (1 - overall_conversion_rate) *
    (1 / after_visit_count + 1 / before_visit_count)
  ) AS z_score,
  CASE WHEN z_score >= 1.96 THEN 'Significant' ELSE 'Not Significant' END AS test_result,
  *
FROM cte_metrics;

### 3.8.2 Critical Analysis

We can see that we still return a significant result even though we already know that there is no other intervention applied or experiment that is running!

This is due to none other than **seasonality** - a very common enemy when it comes to comparing experiment results sequentially. This is exactly one of the reasons why we like to run experiments in the same time frame with split tests.

The seasonality is effectively removed when you compare like-for-like periods as we showed in our A/B test and we can rely on these results much more than just looking at a simple before and after analysis!

## 3.9 Year-on-Year Comparison

Another method to account for seasonality is to compare results for the same period - but for 1 year earlier.

Here lies another danger for experimentation testing - especially in the world of AI and just in general for all technology!

A lot can happen in 1 year - however we also need to confirm to ourselves that there is no seasonality involved when running our experiments.

### 3.9.1 SQL Implementation

Let's illustrate this example by comparing the 3 months where we were running our experiment with the same 3 months of data from 2025 one year earlier.

In [ ]:
%%sql
WITH cte_base AS (
SELECT
    CASE
        WHEN visits.visit_timestamp BETWEEN DATE '2025-01-01' AND DATE '2025-03-31' THEN 'before'
        ELSE 'after'
    END AS experiment_group,
    visits.visit_id,
    visits.user_id,
    CASE WHEN sales.sale_id IS NOT NULL THEN 1 ELSE 0 END AS sale_flag,
    COALESCE(products.price_usd, 0) AS sale_amount
FROM visits
LEFT JOIN sales
    ON visits.visit_id = sales.visit_id
LEFT JOIN products
    ON sales.product_id = products.product_id
WHERE visits.visit_timestamp BETWEEN DATE '2025-01-01' AND DATE '2025-03-31'
OR visits.visit_timestamp BETWEEN DATE '2026-01-01' AND DATE '2026-03-31'
),
cte_before AS (
SELECT
    COUNT(DISTINCT visit_id) AS before_visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS before_sales_count,
    SUM(sale_amount) AS before_sales_amount,
    before_sales_count / before_visit_count AS before_conversion_rate
FROM cte_base
WHERE experiment_group = 'before'
),
cte_after AS (
SELECT
    COUNT(DISTINCT visit_id) AS after_visit_count,
    COUNT(DISTINCT CASE WHEN sale_flag = 1 THEN visit_id ELSE NULL END) AS after_sales_count,
    SUM(sale_amount) AS after_sales_amount,
    after_sales_count / after_visit_count AS after_conversion_rate
FROM cte_base
WHERE experiment_group = 'after'
),
cte_metrics AS (
SELECT
    # uplift metrics
    after_sales_count - before_sales_count AS incremental_sales_count,
    after_sales_amount - before_sales_amount AS incremental_sales_amount,
    100 * ( after_sales_count / before_sales_count - 1 ) AS sales_count_uplift,
    100 * ( after_sales_amount / before_sales_amount - 1 ) AS sales_amount_uplift,
    100 * ( after_conversion_rate / before_conversion_rate - 1 ) AS conversion_uplift,
    before.*,
    after.*
FROM cte_after AS after
CROSS JOIN cte_before AS before
)
SELECT
  (after_sales_count + before_sales_count) /
    (after_visit_count + before_visit_count) AS overall_conversion_rate,
  # numerator
  (after_conversion_rate - before_conversion_rate) /
  # denominator
  SQRT(
    overall_conversion_rate * (1 - overall_conversion_rate) *
    (1 / after_visit_count + 1 / before_visit_count)
  ) AS z_score,
  CASE WHEN z_score >= 1.96 THEN 'Significant' ELSE 'Not Significant' END AS test_result,
  *
FROM cte_metrics;

### 3.9.2 Critical Analysis

This demonstrates how we can compare our analysis periods to account for seasonality and also to perform split testing to analyse uplift directly in the same period for best practice.

Although this isn't as good as running an A/B split test - sometimes using a year-on-year analysis helps us isolate our uplift effects while taking seasonality out of the picture.

# Conclusion

Congratulations!!!

You made it to the end of this tutorial - we covered a lot of content so hopefully this Python/SQL notebook has been very useful and shows you a few ways which we can use SQL to prepare our NLP text data and measure the effectiveness of A/B test experiments